QuickStart: Compare Runs choose a model and deploy it to a rest API
GOALS of this:
-Run a hyperparameter sweep on a training script
-Choose the results of the runs in the MLFlow UI
-Choose the best runs and register it as a Model
-Deploy the model to a Rest API
-Build a container image suitable for deployment to a cloud platform

In [69]:
# !pip install -r ../requirements.txt
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
mlflow.set_tracking_uri('http://127.0.0.1:5000') #any expirements done on mlflow are tracked here.
import mlflow
from mlflow.models import infer_signature

In [62]:
df_raw=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
df_raw

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [70]:
#train test split

train,test = train_test_split(df_raw,test_size=0.2,random_state=28041997)

In [71]:
train_x = train.drop(['quality'],axis=1).values
train_y = train[['quality']].values.ravel()
test_x = test.drop(['quality'],axis=1).values
test_y = test[['quality']].values.ravel()

train_x,valid_x,train_y,valid_y = train_test_split(train_x,train_y,test_size=0.2,random_state=28041997)

sig = infer_signature(train_x,train_y)

In [72]:
def train_model(params,epochs, train_x,train_y,val_x,val_y):
    mean = np.mean(train_x,axis=0)
    var = np.var(train_x,axis=0)
    model = keras.Sequential(
            [
                keras.Input([train_x.shape[1]]),
                keras.layers.Normalization(mean=mean,variance=var),
                keras.layers.Dense(64,activation='relu'),
                keras.layers.Dense(1)
            ])
    model.compile(optimizer=keras.optimizers.SGD(learning_rate=params['lr'],momentum=params['momentum']),
        loss = 'mean_squared_error',
        metrics = [keras.metrics.RootMeanSquaredError()]
    )
    
    ## Train the ANN model with lr and momentum params with MLFlow tracking
    with mlflow.start_run(nested = True):
        model.fit(train_x,train_y,validation_data = (val_x,val_y), epochs = epochs,batch_size=64)
        eval_result = model.evaluate(val_x,val_y, batch_size=64)
        eval_rmse = eval_result[1]
        mlflow.log_params(params)
        mlflow.log_metric('eval_rmse',eval_rmse)
        mlflow.tensorflow.log_model(model,'model',signature=sig)
    
        return {'loss':eval_rmse,'status': STATUS_OK, 'model':model}

In [73]:
def objective(params):
    result = train_model(
        params,
        epochs =3,
        train_x = train_x,
        train_y = train_y,
        val_x = valid_x,
        val_y= valid_y
    )
    return result


In [74]:
space = {
    'lr':hp.loguniform('lr',np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [75]:
mlflow.set_experiment('wine_quality')
with mlflow.start_run():
    trials = Trials()
    best = fmin(
        fn = objective,
        space = space,
        algo = tpe.suggest,
        max_evals=4,
        trials=trials
    )
    best_run = sorted(trials.results, key=lambda x: x['loss'])[0]
    mlflow.log_params(best)
    mlflow.log_metric('eval_rmse', best_run['loss'])
    mlflow.tensorflow.log_model(best_run['model'],'model',signature=sig)

    print(f'Best parameters: {best}')
    print(f"Best Eval RMSE: {best_run['loss']}")

2025/12/20 17:29:18 INFO mlflow.tracking.fluent: Experiment with name 'wine_quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

49/49 [==============================] - 0s 4ms/step - loss: 32.1375 - root_mean_squared_error: 5.6690 - val_loss: 29.9579 - val_root_mean_squared_error: 5.4734

Epoch 2/3                                            

49/49 [==============================] - 0s 1ms/step - loss: 28.2763 - root_mean_squared_error: 5.3175 - val_loss: 26.3651 - val_root_mean_squared_error: 5.1347

Epoch 3/3                                            

49/49 [==============================] - 0s 1ms/step - loss: 24.8874 - root_mean_squared_error: 4.9887 - val_loss: 23.1966 - val_root_mean_squared_error: 4.8163

13/13 [==============================] - 0s 2ms/step - loss: 23.1966 - root_mean_squared_error: 4.8163

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]INFO:tensorflow:Assets written to: C:\Users\theVa\AppData\Local\Temp\tmpqwd14b6b\model\data\model\assets
Epoch 1/3                                                                     

49/49 [